In [ ]:
!gdown --id 1SA8RmNwnN4zTDRRvKjkvXWaFrSpk22o0

Downloading...
From: https://drive.google.com/uc?id=1htCJ4b2rFiifPvEbo0GJpP1x8ZwCOS1v
To: /content/drive/My Drive/USP/projeto/artigos/ENIAC-10-GDELT/Datasets/with_outliers/Events/Events.zip
199MB [00:08, 23.4MB/s]


In [168]:
!unzip Events.zip

Archive:  Events.zip
replace Events/Corruption.plk? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Events/Corruption.plk   
  inflating: Events/Inflation.plk    
  inflating: Events/Racism.plk       
  inflating: Events/Agriculture.plk  
  inflating: Events/Earthquake.plk   
  inflating: Events/Tsunami.plk      
  inflating: Events/Covid.plk        
  inflating: Events/Terrorism.plk    
  inflating: Events/War.plk          
  inflating: Events/Immigration.plk  


In [169]:
from pathlib import Path
import pandas as pd

def load_dataset(all_one_dataset):
    datasets_dictionary = {}
    if all_one_dataset == 'All':
        basepath = Path('./Events/')
        files_in_basepath = basepath.iterdir()
        for item in files_in_basepath:
            if item.is_file():
                df = pd.read_pickle('./Events/' + item.name)
                datasets_dictionary[item.name.replace('.plk', '')] = df
    else:
        df = pd.read_pickle('./Events/' + all_one_dataset + '.plk')
        datasets_dictionary[all_one_dataset] = df

    return datasets_dictionary

In [170]:
dfs = load_dataset('All')

In [ ]:
dfs['Racism']

In [ ]:
from sklearn.model_selection import train_test_split
def create_outliers(datasets_dictionary, dataset, outliers_percent):
    df_out = pd.DataFrame(columns=list(datasets_dictionary[dataset].columns))

    for ds in datasets_dictionary.keys():
        if ds != dataset:
            outliers, _ = train_test_split(datasets_dictionary[ds], train_size=outliers_percent, random_state=42,
                                           shuffle=True)
            df_out = df_out.append(outliers)
    
    return df_out

In [ ]:
def concatenate_outliers(datasets_dictionary, outliers_percent):
    new_dfs = {}

    for dataset in datasets_dictionary.keys():
        df_out = create_outliers(datasets_dictionary, dataset, outliers_percent)
        new_dfs[dataset] = datasets_dictionary[dataset].append(df_out)
        new_dfs[dataset]['class'] = ['interest'] * 6000 + ['outlier'] * len(df_out)
        
    return new_dfs

In [ ]:
new_dfs = concatenate_outliers(dfs,0.075)

In [182]:
new_dfs['Immigration']

,Id,Date,Title,Location,DBERTML,lat_long,class
0,20190922180000-955,2019-09-22,A looming change in immigration policy is spoo...,"[3#Santa Clara County, California, United Stat...","[-0.058543608, 0.048537247, 0.04762246, -0.000...","[37.7749, -122.419]",interest
1,20190922211500-572,2019-09-22,"U.S. Rep. Joe Kennedy, meeting Worcester LGBTQ...","[1#Jamaica#JM#JM#18.25#-77.5#JM, 1#United Stat...","[-0.024511885, 0.055219326, 0.105707966, -0.05...","[2, 33]",interest
2,20190922204500-955,2019-09-22,Some Latinos back Trump's immigration policies...,"[1#Mexico#MX#MX#23#-102#MX, 2#Arizona, United ...","[-0.04792244, -0.011049615, -0.00081020297, -0...","[28.3667, -100.317]",interest
3,20190922163000-T20,2019-09-22,Immigration : pourquoi Macron durcit le ton,[1#France#FR#FR#46#2#FR],"[-0.026617987, -0.0042958944, -0.019341556, -0...","[46, 2]",interest
4,20190922163000-T100,2019-09-22,Immigration : la couverture sant&#xE9; des mig...,[1#France#FR#FR#46#2#FR],"[-0.048499078, 0.0015756552, -0.0020941587, -0...","[46, 2]",interest
...,...,...,...,...,...,...,...
3772,20200320200000-454,2020-03-20,Education miinister's response to report on an...,"[4#Caledon, Ontario, Canada#CA#CA08#43.8667#-8...","[-0.016333843, -0.01781449, 0.03411332, -0.060...","[43.8667, -80]",outlier
5191,20200601221500-1920,2020-06-01,How government priorities favor racism over ci...,"[2#Pennsylvania, United States#US#USPA#40.5773...","[-0.014070964, 0.060328774, -0.015437853, -0.0...","[39.9523, -75.1638]",outlier
5226,20200601021500-838,2020-06-01,Sudbury demonstration denounces racism in U.S....,"[2#Georgia, United States#US#USGA#32.9866#-83....","[0.0043586143, -0.03171569, -0.007973451, -0.0...","[43.6667, -79.4167]",outlier
5390,20200602061500-907,2020-06-02,'Hate just hides': Biden vows to take on syste...,"[1#United States#US#US#39.828175#-98.5795#US, ...","[0.026523655, -0.0064231325, 0.05273414, -0.06...","[41.85, -87.6501]",outlier


In [ ]:
for dataset in new_dfs.keys():
  new_dfs[dataset].to_pickle(dataset + '.plk')